---
<div align="center">

# LSDS | Machine Learning Pipeline
</div>

---

> ADD PROJECT OVERVIEW

---
## Project Dependencies
---

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Remove Warnings
import warnings
warnings.filterwarnings('ignore')

---
---
---

## Testing pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
# spark.read.csv('./Datasets/CHARTEVENTS.csv', header=True).show()
spark.read.csv('./Machine-Learning-Pipeline/Datasets/CHARTEVENTS.csv', header=True)

DataFrame[ROW_ID: string, SUBJECT_ID: string, HADM_ID: string, ICUSTAY_ID: string, ITEMID: string, CHARTTIME: string, STORETIME: string, CGID: string, VALUE: string, VALUENUM: string, VALUEUOM: string, WARNING: string, ERROR: string, RESULTSTATUS: string, STOPPED: string]

In [7]:
import pandas as pd
from pyspark.sql.functions import pandas_udf

@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    # Simply plus one by using pandas Series.
    return series + 1